# Import Library

In [ ]:
# !pip install datasets transformers pandas 
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install scikit-learn
# !pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 62.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 66.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.1 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━

In [6]:
from datasets import load_dataset
import datasets
from transformers import BertModel, BertTokenizer, PhobertTokenizer, AutoModel , AutoTokenizer , AutoModelForSequenceClassification
import torch
import torch.nn as nn
from pyvi.ViTokenizer import tokenize
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DataCollatorWithPadding
import tqdm
import random
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report
import os
from sklearn.decomposition import PCA
pd.set_option('display.max_colwidth', None)

/home/user/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random_seed = 42
torch.manual_seed(random_seed)
random_vector = torch.randn(5)
print(random_vector)

tensor([ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229])


# Config

In [62]:
VIETNAM_MODEL = 'vinai/phobert-base-v2'
ENGLISH_MODEL = 'cardiffnlp/twitter-roberta-base-sentiment'
DATA_PATH = "/home/user/THESIS_CODE_REPO/data/pretrain_data/phomt.csv"
SOCIAL_DATA_PATH = "/home/user/THESIS_CODE_REPO/data/pretrain_data/social_pretrain_data.csv"
SAVE_PATH = "/home/user/THESIS_CODE_REPO/model checkpoint/best_model3.pth"
VIOCD_DATA_PATH = "tarudesu/ViOCD"
VLSP_DATA_PATH = "/home/user/THESIS_CODE_REPO/data/testing_data/vlsp/test_raw_ANS.txt"
AIVIVN_DATA_PATH = "/home/user/THESIS_CODE_REPO/data/testing_data/aivivn/aivivn_test.csv"
NTC_SCV = "thainq107/vi-ntc-scv"
UIT_VSFC_0 = "/home/user/THESIS_CODE_REPO/data/testing_data/uit_vsfc/translated_test_dataset_0.csv"
UIT_VSFC_1 = "/home/user/THESIS_CODE_REPO/data/testing_data/uit_vsfc/translated_test_dataset_1.csv"
UIT_VSFC_2 = "/home/user/THESIS_CODE_REPO/data/testing_data/uit_vsfc/translated_test_dataset_2.csv"
MATRIX_PATH = "/home/user/THESIS_CODE_REPO/english_embedding_compress/matrix.pt"
MAX_LEN = 50
BATCH_SIZE = 128
VIET_HIDDEN_DIM = 768
GPU_DEVICE = 0
POS_DATA_SIZE = 50000
NEU_DATA_SIZE = 50000          
NEG_DATA_SIZE = 50000
vietnamese_model_embedding_dim = 768 
english_embedding_dim = 768
intermediate_dim = 768 

In [9]:
torch.cuda.set_device(GPU_DEVICE)
print(torch.cuda.current_device())  # Output: 0 (selected GPU ID)

0


# 1. Data Loading

In [10]:
data = pd.read_csv (DATA_PATH)
data = data.sample (500000, random_state = 42)

In [12]:
neg_data = data[data.label == 0].sample (NEG_DATA_SIZE, random_state = 42)
neu_data = data[data.label == 1].sample (NEU_DATA_SIZE, random_state = 42)
pos_data = data[data.label == 2].sample (POS_DATA_SIZE, random_state = 42)

balance_data = pd.concat ([neg_data, neu_data, pos_data])
balance_data.label.value_counts()

label
0    50000
1    50000
2    50000
Name: count, dtype: int64

In [13]:
balance_data = balance_data.applymap (lambda x: x.replace(" .", "") if isinstance (x, str) else x)
balance_data = balance_data.applymap (lambda x: x.replace("\n", "") if isinstance (x, str) else x)
balance_data['EnglishSentences'] = balance_data['EnglishSentences'] + "."

/tmp/ipykernel_7216/1164202334.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  balance_data = balance_data.applymap (lambda x: x.replace(" .", "") if isinstance (x, str) else x)
/tmp/ipykernel_7216/1164202334.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  balance_data = balance_data.applymap (lambda x: x.replace("\n", "") if isinstance (x, str) else x)


In [14]:
balance_data

,Unnamed: 0,EnglishSentences,VietnameseSentences,label
314177,230557,"There is no life possible at the surface of Mars today , but it might still be hiding underground.","Ngày nay , không thể có sự sống trên bề mặt sau Hoả , nhưng nó có thể che dấu một thế giới ngầm",0
178286,104045,They did n't want to send their books.,Họ không muốn gửi sách,0
134765,965888,I do n't want to come.,Con không muốn đi,0
124776,1198803,You will not believe what I...,Chị sẽ không tin chuyện em,0
269284,2565456,"Although it was widely used in the past , the drug has mostly been discontinued and hence is now mostly no longer available.","Mặc dù nó đã được sử dụng rộng rãi trong quá khứ , loại thuốc này hầu hết đã bị ngưng sử dụng và do đó hiện nay hầu như không còn nữa",0
...,...,...,...,...
340834,339690,"Drinking coconut water , which contains potassium and magnesium , can help with normal muscle function.","Uống nước dừa , Trong nước dừa có hàm lượng kali và ma-giê cao giúp điều hoà chức năng cơ bắp",2
261312,1223140,We 'll seize Munich.,Chúng ta sẽ chiếm lấy Munich,2
437976,2169170,We may actually have a shot at finding the city.,Ta sẽ có cơ hội tìm ra thành phố đó,2
253651,2021627,Respect 'em from the start.,Luật là phải tôn trọng,2


In [15]:
social_data = pd.read_csv (SOCIAL_DATA_PATH)
# rename the columns VietnameseText and EnglishText to VietnameseSentences and EnglishSentences
social_data = social_data.rename (columns = {"VietnameseText": "VietnameseSentences", "EnglishText": "EnglishSentences" , "Label" : "label"})
social_data

,VietnameseSentences,EnglishSentences,label,NegProb,NeuProb,PosProb
0,"Dậy sớm để thành công, má dặn rồi","Wake up early to succeed, just like mom said.",2,0.018437,0.274801,0.706762
1,Không muốn chửi đâu nhưng mà Đ cụ bọn ViB.,"Don’t want to curse, but fuck ViB.",0,0.911908,0.076529,0.011563
2,Ca nít ranh này có cc gì để tư vấn,What the hell does this little brat have to advise?,0,0.968718,0.027846,0.003436
3,"có hẳn tên giai điệu riêng luôn, quá chất's","It even has its own melody name, so damn cool.",2,0.002914,0.019273,0.977813
4,Sao a lại kể chuyện của e cho mn thế,Why did you tell everyone my story?,0,0.632275,0.351856,0.015870
...,...,...,...,...,...,...
74666,"Vâng, em cảm ơn 2 bác, chúc 2 bác an lành nha.","Yes, I thank you both, and I wish you peace and well-being.",2,0.001499,0.022456,0.976045
74667,"Vãi cả cái sân, svd quốc gia nhìn nhục vcl","Damn, the whole field, the national stadium looks so shameful.",0,0.977510,0.020381,0.002109
74668,T xin dừng cuộc chơi ở 11 Pro Max,I’m calling it quits at the 11 Pro Max.,1,0.220230,0.693460,0.086310
74669,Hàng thủ hqua chả cho Tấn Trường solo luôn,The defense yesterday didn’t even let Tấn Trường solo.,0,0.752859,0.230803,0.016337


In [16]:
pretrain_data = pd.concat ([balance_data, social_data])
pretrain_data.label.value_counts()

label
0    75000
1    75000
2    74671
Name: count, dtype: int64

# 2. Processing Data

In [17]:
id2label = {0: "NEGATIVE", 1 : "NEUTRAL" , 2: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 2 , "NEUTRAL" : 1}

vietnamese_tokenizer = AutoTokenizer.from_pretrained(VIETNAM_MODEL)
vietnamese_model = AutoModel.from_pretrained(VIETNAM_MODEL).to(device)

english_tokenizer = AutoTokenizer.from_pretrained(ENGLISH_MODEL)
english_model = AutoModelForSequenceClassification.from_pretrained(ENGLISH_MODEL, id2label = id2label, label2id = label2id).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 2.1 Tokenize Data

In [18]:
# Tokenize sentence
def tokenize_sentences(english_sentences, vietnamese_sentences, vietnamese_tokenizer, english_tokenizer, seed=42):
    # Tokenize English sentences and generated positives/negatives
    english_sentences_tokenized = english_tokenizer(english_sentences, padding="max_length", max_length=MAX_LEN, truncation=True)

    # Tokenize Vietnamese sentences
    vietnamese_sentences_segmented = [tokenize(sentence) for sentence in vietnamese_sentences]
    vietnamese_sentences_tokenized = vietnamese_tokenizer(vietnamese_sentences_segmented, padding="max_length", max_length=MAX_LEN, truncation=True)
    
    return {
        "english_sentences_input_ids": english_sentences_tokenized["input_ids"],
        "english_sentences_attention_mask": english_sentences_tokenized["attention_mask"],
        "vietnamese_sentences_input_ids": vietnamese_sentences_tokenized["input_ids"], 
    }
def extract_matrice_embedding(input_ids, model , is_english = False):
    if is_english :
        with torch.no_grad():
            first_layer_embeddings = model.roberta.embeddings(input_ids.to(model.device))
    else :
        with torch.no_grad():
            first_layer_embeddings = model.embeddings(input_ids.to(model.device))
    return first_layer_embeddings.cpu()

In [19]:
translation_dataset_tokenized = tokenize_sentences(pretrain_data["EnglishSentences"].to_list(),
                                                   pretrain_data["VietnameseSentences"].to_list(),
                                                   vietnamese_tokenizer,
                                                   english_tokenizer)

## 2.2 Dataloader

In [20]:
# Custom dataset
class ContrastiveLearningDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset["english_sentences_input_ids"])

    def __getitem__(self, index):
        english_sentence_input_ids = torch.tensor(self.dataset["english_sentences_input_ids"][index], dtype = torch.long)
        english_sentence_attention_mask = torch.tensor(self.dataset["english_sentences_attention_mask"][index], dtype = torch.long)
        vietnamese_sentence_input_ids  = torch.tensor(self.dataset["vietnamese_sentences_input_ids"][index], dtype = torch.long)
        return {
            "english_sentence_input_ids": english_sentence_input_ids,
            "vietnamese_sentence_input_ids": vietnamese_sentence_input_ids,
            "english_sentence_attention_mask":english_sentence_attention_mask
        }

In [21]:
# Split dataset
def split_dataset(dataset):
    train_size = int(0.7 * len(dataset))  
    dev_size = int(0.15 * len(dataset))    
    test_size = len(dataset) - train_size - dev_size 
    torch.manual_seed(42)
    train_dataset, dev_dataset, test_dataset = random_split(dataset, [train_size, dev_size, test_size])
    return train_dataset, dev_dataset, test_dataset

In [22]:
custom_translation_dataset = ContrastiveLearningDataset(translation_dataset_tokenized)
train_dataset, val_dataset, test_dataset = split_dataset(custom_translation_dataset)

### Bug Checking

In [23]:
len(custom_translation_dataset) , len (train_dataset), len (val_dataset) , len(test_dataset)

(224671, 157269, 33700, 33702)

In [24]:
data = next(iter(custom_translation_dataset))
english_id = data["english_sentence_input_ids"]
vi_id = data["vietnamese_sentence_input_ids"]
att_mask_eng = data["english_sentence_attention_mask"]
print(vietnamese_tokenizer.decode (vi_id))
print (english_tokenizer.decode(english_id))
print (att_mask_eng)

<s> Ngày_nay , không_thể có sự sống trên bề_mặt sau Hoả , nhưng nó có_thể che dấu một thế_giới ngầm </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<s>There is no life possible at the surface of Mars today , but it might still be hiding underground.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])


## 2.3 extract ids, logits sentiment

In [25]:
# Custom collate function to handle padding within each batch
def collate_fn(batch):
    global english_tokenizer, vietnamese_tokenizer, english_model
    english_sentences_input_ids = [sample["english_sentence_input_ids"] for sample in batch]
    vietnamese_sentences_input_ids  = [sample["vietnamese_sentence_input_ids"] for sample in batch]
    
    english_sentences_attention_mask = [sample["english_sentence_attention_mask"] for sample in batch]
    # Padding to the maximum length in each batch using DataCollatorWithPadding from transformers
    # Ensure we pass the respective tokenizer for padding
    eng_padding_collator = DataCollatorWithPadding(tokenizer=english_tokenizer, padding=True, return_tensors='pt')
    vi_padding_collator = DataCollatorWithPadding(tokenizer=vietnamese_tokenizer, padding=True, return_tensors='pt')
    # collate the input_ids
    english_sentences_batch = eng_padding_collator({'input_ids': english_sentences_input_ids,"attention_mask" : english_sentences_attention_mask}).to(device)
    vietnamese_sentence_batch = vi_padding_collator({'input_ids': vietnamese_sentences_input_ids})

    
    vietnamese_embedding = extract_matrice_embedding(vietnamese_sentence_batch['input_ids'], vietnamese_model , is_english = False).to(device)
    
    with torch.no_grad():
            
            outputs = english_model(input_ids=english_sentences_batch['input_ids'], attention_mask=english_sentences_batch['attention_mask'])
            english_logits = outputs.logits
            english_prediction = torch.argmax(english_logits, dim=1)
            
    
    return {
        'vietnamese_sentence_embeddings': vietnamese_embedding,
        #'english_sentence_embeddings':english_embedding,
        'english_logits' : english_logits,
        'english_prediction' : english_prediction,

        # This is for the debug
        'english_id' : english_sentences_batch['input_ids'] ,
        'vi_id' : vietnamese_sentence_batch['input_ids']
    }

# Prepare the dataloader for training
def prepare_dataloader(dataset, batch_size):
    return DataLoader(dataset, batch_size = batch_size, shuffle = False, collate_fn=collate_fn)

In [26]:
train_dataloader = prepare_dataloader(train_dataset, BATCH_SIZE)
val_dataloader = prepare_dataloader(val_dataset, BATCH_SIZE)

### Bug checking

In [27]:
first_batch = next(iter(train_dataloader))

In [28]:
index = 0
english_id = first_batch["english_id"][index]
vi_id = first_batch["vi_id"][index]

print(vietnamese_tokenizer.decode (vi_id))
print (english_tokenizer.decode(english_id))
print (first_batch['english_logits'][index])
print (first_batch['english_prediction'][index])

<s> loại người như mày tao ko có gì để nói . hết </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<s>I have nothing to say to people like you. That's it.  </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
tensor([ 1.3718,  0.4637, -1.9053], device='cuda:0')
tensor(0, device='cuda:0')


# 3. Model and Train

## 3.1 Define the mapper architecure

In [29]:
class EmbeddingAlignmentModel(nn.Module):
    def __init__(self, 
                 vietnamese_embedding_dim, 
                 english_embedding_dim, 
                 intermediate_dim, 
                 distilled_english_embeddings,
                 num_attention_heads=4):
        """
        Model to align Vietnamese embeddings to distilled English embeddings.

        Args:
            vietnamese_embedding_dim (int): Dimensionality of Vietnamese embeddings.
            english_embedding_dim (int): Dimensionality of English embeddings.
            intermediate_dim (int): Dimensionality of the intermediate space.
            distilled_english_embeddings (torch.Tensor): Distilled English embeddings.
            num_attention_heads (int): Number of heads in the multi-head attention mechanism.
        """
        super(EmbeddingAlignmentModel, self).__init__()
        
        # Input Transformation Layer
        self.input_transform = nn.Linear(vietnamese_embedding_dim, intermediate_dim)
        self.dropout = nn.Dropout(p=0.1)  # Dropout to prevent overfitting
        
        # Cross-Language Reprogramming Layer
        self.cross_attention = nn.MultiheadAttention(embed_dim=intermediate_dim, 
                                                     num_heads=num_attention_heads, 
                                                     batch_first=True)
        
        # Distilled English embeddings
        self.register_buffer("distilled_embeddings", distilled_english_embeddings)
        
        # Output Projection to English embedding space
        self.output_project = nn.Linear(intermediate_dim, english_embedding_dim)

        # Normalization Layer
        self.output_normalization = nn.LayerNorm(english_embedding_dim)

    def forward(self, vietnamese_embeddings):
        """
        Forward pass for embedding alignment.

        Args:
            vietnamese_embeddings (torch.Tensor): Input embeddings from the Vietnamese model (batch_size, seq_len, embed_dim).

        Returns:
            torch.Tensor: Aligned embeddings in the distilled English embedding space.
        """
        # Step 1: Transform Vietnamese embeddings to intermediate space
        transformed_embeddings = self.input_transform(vietnamese_embeddings)
        transformed_embeddings = self.dropout(transformed_embeddings)
        batch_size, seq_len, _ = transformed_embeddings.size()
        expanded_embeddings = self.distilled_embeddings.unsqueeze(0).expand(batch_size, -1, -1)  # (batch_size, n_clusters, embedding_dim)

        # Step 2: Cross-attention with distilled embeddings as keys/values
        
        reprogrammed_embeddings, _ = self.cross_attention(
            query=transformed_embeddings,
            key=expanded_embeddings,
            value=expanded_embeddings
        )

        # Step 3: Project to English embedding space
        aligned_embeddings = self.output_project(reprogrammed_embeddings)

        # Step 4: Normalize the aligned embeddings
        normalized_embeddings = self.output_normalization(aligned_embeddings)

        return normalized_embeddings

## 3.2 For Loop For Training

In [30]:
import torch
from tqdm.auto import tqdm
import os

def train_mapper(model, optimizer, criterion, train_dataloader, val_dataloader, num_epochs=10, english_model=None, n_epochs_stop=5, save_path="best_model.pth"):
    """
    Train the Mapper model with early stopping and auto-saving.

    Args:
        model: The Mapper model to train.
        optimizer: Optimizer for training.
        criterion: Loss function.
        train_dataloader: DataLoader for training data.
        val_dataloader: DataLoader for validation data.
        num_epochs (int): Maximum number of epochs.
        english_model: Pretrained frozen English model.
        n_epochs_stop (int): Number of epochs to wait for improvement before stopping.
        save_path (str): Path to save the best model.
    """
    # Ensure the English model is frozen
    for param in english_model.parameters():
        param.requires_grad = False

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0.0

        # Training phase
        for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
            vietnamese_embedding = batch['vietnamese_sentence_embeddings']
            english_prediction = batch['english_prediction']

            # Forward pass through the mapper
            mapped_embedding = model(vietnamese_embedding)

            # Forward pass through the frozen English model
            first_layer_embeddings = english_model.roberta.encoder(mapped_embedding)[0]
            fake_english_logits = english_model.classifier(first_layer_embeddings)

            # Compute loss
            #print ("english_prediction, fake_english_logits", english_prediction, fake_english_logits)
            loss = criterion(fake_english_logits,english_prediction)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_dataloader)

        # Validation phase
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validating"):
                vietnamese_embedding = batch['vietnamese_sentence_embeddings']
                english_prediction = batch['english_prediction']

                # Forward pass through the mapper
                mapped_embedding = model(vietnamese_embedding)

                # Forward pass through the frozen English model
                first_layer_embeddings = english_model.roberta.encoder(mapped_embedding)[0]
                fake_english_logits = english_model.classifier(first_layer_embeddings)

                # Compute validation loss
                val_loss = criterion(fake_english_logits, english_prediction )
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)

        print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Early stopping and model saving
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Save the model
            torch.save(model.state_dict(), save_path)
            print(f"New best model saved with Val Loss: {best_val_loss:.4f}")
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= n_epochs_stop:
            print("Early stopping triggered. Training stopped.")
            break

    # Load the best model after training
    model.load_state_dict(torch.load(save_path))
    print(f"Training completed. Best Val Loss: {best_val_loss:.4f}. Model reloaded from {save_path}.")
    return model

In [31]:
from sklearn.cluster import KMeans

def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

def distill_embeddings_kmeans(english_embeddings, n_clusters=1000):
    """Distill English embeddings using k-means clustering."""
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clustered_embeddings = kmeans.fit(english_embeddings.cpu().numpy())
    centroids = torch.tensor(clustered_embeddings.cluster_centers_, dtype=torch.float32)
    return centroids
    
# Example usage:
# model = MapperModel(768, 768).to(device)

# pretrained_english_embeddings = english_model.roberta.embeddings.word_embeddings.weight.detach()
# distilled_embeddings = distill_embeddings_kmeans (pretrained_english_embeddings, n_clusters = 10000)
#distilled_embeddings = torch.load ("/kaggle/input/embedding-matrice-distil/distilled_embeddings.pt", weights_only=True)

In [27]:
# pretrained_english_embeddings = english_model.roberta.embeddings.word_embeddings.weight.detach()
# distilled_embeddings = distill_embeddings_gpu_kmeans(pretrained_english_embeddings, n_clusters=10000)

In [34]:
import torch
import faiss
import numpy as np
from tqdm import tqdm

def distill_embeddings_faiss_kmeans(english_embeddings, n_clusters=1000):
    """Correct FAISS K-means implementation with progress tracking"""
    # Convert embeddings to numpy array
    X = english_embeddings.cpu().numpy().astype('float32')
    d = X.shape[1]  # Dimension of embeddings
    
    # Initialize FAISS K-means parameters
    kmeans = faiss.Kmeans(
        d, 
        n_clusters,
        niter=300,                # Number of iterations
        nredo=3,                  # Number of redo iterations
        verbose=False,            # Disable FAISS internal printing
        gpu=True,                 # Enable GPU acceleration
        seed=42,                  # For reproducibility
        max_points_per_centroid=100000
    )
    
    # Create progress bar
    with tqdm(total=300, desc="FAISS K-means") as pbar:
        # Custom callback function
        def progress_callback(epoch):
            pbar.update(1)
            pbar.refresh()
        
        # Assign callback to FAISS instance
        kmeans.progress_callback = progress_callback
        
        # Train the model
        kmeans.train(X)
    
    return torch.tensor(kmeans.centroids, device=english_embeddings.device)

In [ ]:
# pretrained_english_embeddings = english_model.roberta.embeddings.word_embeddings.weight.detach()
# distilled_embeddings = distill_embeddings_faiss_kmeans(pretrained_english_embeddings, n_clusters=1000)

FAISS K-means:   0%|          | 0/300 [00:00<?, ?it/s]

FAISS K-means:   0%|          | 0/300 [18:14<?, ?it/s]


In [51]:
#torch.save(distilled_embeddings,MATRIX_PATH )
# Later, load the matrix back
distilled_embeddings = torch.load(MATRIX_PATH)

In [52]:

# Initialize the alignment model with pretrained embeddings
model = EmbeddingAlignmentModel(
    vietnamese_embedding_dim=vietnamese_model_embedding_dim,
    english_embedding_dim=english_embedding_dim,
    intermediate_dim=intermediate_dim,
    distilled_english_embeddings=distilled_embeddings,
    #distilled_english_embeddings=pretrained_english_embeddings,
    num_attention_heads=8
).to(device)

model.apply(initialize_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [38]:
def count_parameters(model):
    """
    Count the total number of parameters in a PyTorch model.
    
    Args:
        model (nn.Module): The PyTorch model instance.
        
    Returns:
        int: The total number of parameters.
        int: The number of trainable parameters.
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

# Example usage:
total_params, trainable_params = count_parameters(model)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 3545088
Trainable parameters: 3545088


In [39]:
trained_mapper = train_mapper(
    model=model, 
    optimizer=optimizer,
    criterion = criterion,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    english_model=english_model,
    num_epochs=10,
    n_epochs_stop=3, save_path= SAVE_PATH
)

Validating: 100%|██████████| 264/264 [00:57<00:00,  4.63it/s]


Epoch 1, Train Loss: 0.9900, Val Loss: 0.9141


RuntimeError: Parent directory Model_Checkpoint does not exist.

# 4. Testing Inference

In [54]:
english_model = AutoModelForSequenceClassification.from_pretrained(ENGLISH_MODEL, id2label = id2label, label2id = label2id).to(device)

In [55]:
first_batch = next(iter(val_dataloader))

In [56]:
index = 0
english_logits = first_batch["english_logits"][index]
vi_id = first_batch["vi_id"][index]
english_id = first_batch['english_id'][index]

print ( "VietNam Sentence ----------- " , vietnamese_tokenizer.decode (vi_id) )
print ( "English Sentence ----------- " , english_tokenizer.decode (english_id) )

vietnamese_embeddings = extract_matrice_embedding(vi_id.view(1,-1), vietnamese_model ,  is_english = False).to(device)
# english_embeddings = extract_matrice_embedding(english_id.view(1,-1), english_model ,  is_english = True).to(device)
# Map Vietnamese embeddings to English space
with torch.no_grad () :
    mapped_vietnamese_embeddings = model(vietnamese_embeddings)

VietNam Sentence -----------  <s> Trong mười giây nữa các bạn sẽ nhập niết_bàn và hoàn_toàn được khai sáng </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
English Sentence -----------  <s>In ten seconds you will reach nirvana and total spiritual enlightenment.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [57]:
encoder_outputs_fake = english_model.roberta.encoder(mapped_vietnamese_embeddings)[0] #
# # 6. Pass the CLS representation through the classification head to get logits
logits_fake = english_model.classifier(encoder_outputs_fake)
# print ( "Fake logits: " , logits_fake )
print ("Fake logits: " , logits_fake)


print ( "Real Logits: " , english_logits )

Fake logits:  tensor([[-0.2891,  1.5106, -1.2190]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
Real Logits:  tensor([-3.0659,  1.2550,  2.0648], device='cuda:0')


In [58]:
# F.cosine_similarity(encoder_outputs_fake, encoder_outputs_real, dim=-1).mean()

# 5. Mapper Sentiment Analysis

## 5.1 Evaluation Function

In [59]:
# Step 5: Use English Model for Sentiment Analysis
from sklearn.metrics import classification_report

def read_txt_to_data(file_path):
    """
    Reads the given TXT file and converts it into a list of dictionaries with 'text' and 'label'.
    """
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    text = []
    for line in lines:
        line = line.strip()
        if line in {"POS", "NEG", "NEU"}:
            # Map labels to indices
            label = {"NEG": 0, "NEU": 1, "POS": 2}[line]
            data.append({"text": " ".join(text).strip(), "label": label})
            text = []  # Reset text for the next entry
        else:
            text.append(line)
    return data

def sentiment_analysis(english_model, english_embeddings):
    # Pass the mapped embeddings through the English model for sentiment classification
    encoder_outputs = english_model.roberta.encoder (english_embeddings)[0]
    logits = english_model.classifier(encoder_outputs)
    #print (logits)
    predicted_class = torch.argmax(logits, dim=-1)
    return predicted_class.item()
    
def tokenize_vietnamese(text):
    # Tokenize the Vietnamese sentence using PhoBERT tokenizer
    vietname_segment = tokenize (text)
    tokens = vietnamese_tokenizer( vietname_segment , return_tensors='pt', truncation=True, padding=True, max_length = MAX_LEN)
    return tokens['input_ids']


# Step 5: Use English Model for Sentiment Analysis
def sentiment_analysis_2class (english_model, english_embeddings):
    """
    Perform sentiment analysis using the English model and handle 3-label output.
    Select between labels 0 (negative) and 2 (positive) based on logits.
    """
    # Pass the mapped embeddings through the English model for sentiment classification
    encoder_outputs = english_model.roberta.encoder(english_embeddings)[0]
    logits = english_model.classifier(encoder_outputs)
    
    # Extract logits for labels 0 and 2
    logits_0 = logits[:, 0]  # Logit for label 0 (negative)
    logits_2 = logits[:, 2]  # Logit for label 2 (positive)
    
    # Select the label with the higher logit
    predicted_class = 0 if logits_0 > logits_2 else 2
    return predicted_class


# Step 6: Sentiment Analysis Pipeline
def sentiment_analysis_pipeline(vietnamese_text, vietnamese_model, mapper, english_model, num_label = 3):
    # 1. Tokenize the Vietnamese text
    input_ids = tokenize_vietnamese(vietnamese_text)
    #print (input_ids)
    # 2. Extract Vietnamese embeddings (value and positional embeddings)
    vietnamese_embeddings = extract_matrice_embedding(input_ids, vietnamese_model, is_english = False).to(device)
    #print (vietnamese_embeddings)
    # 3. Map the Vietnamese embeddings to the English embedding space
    with torch.no_grad () :
        mapped_vietnamese_embeddings = mapper(vietnamese_embeddings)
    #print (mapped_vietnamese_embeddings)
    # 4. Perform sentiment analysis using the English model
    if num_label == 3 :
        sentiment_output = sentiment_analysis(english_model, mapped_vietnamese_embeddings)
    elif num_label == 2 :
        sentiment_output = sentiment_analysis_2class(english_model, mapped_vietnamese_embeddings)
    return sentiment_output


def reverse_labels(dataset):
    def map_label(example):
        if example['label'] == 0:
            return {'label': 2}  # Change positive (0) to 2
        elif example['label'] == 1:
            return {'label': 0}  # Change negative (1) to 0
        return example

    return dataset.map(map_label)

def map_ntc_scv_label (dataset) :
    def map_label(example):
        if example['label'] == 1:
            return {'label': 2}
        elif example['label'] == 0:
            return {'label': 0}
        return example
    return dataset.map(map_label)   

def evaluate_model(dataset, vietnamese_model, mapper, english_model, text_col , label_col, num_label = 3):
    """
    Evaluate the model on the given dataset.
    """
    true_labels = []
    predicted_labels = []

    for example in tqdm(dataset):
        vietnamese_text = example[text_col]
        true_label = example[label_col]

        # Predict sentiment using the pipeline
        predicted_label = sentiment_analysis_pipeline(vietnamese_text, vietnamese_model, mapper, english_model, num_label)

        true_labels.append(true_label)
        predicted_labels.append(predicted_label)
    if num_label == 3 :
        report = classification_report(true_labels, predicted_labels, target_names=["negative", "neutral", "positive"], digits=4)
    elif num_label == 2 :
        report = classification_report(true_labels, predicted_labels, target_names=["negative", "positive"], digits=4)
    return report

# Step 2: Convert to Hugging Face Dataset
def create_hf_dataset(data):
    """
    Converts a list of dictionaries into a Hugging Face dataset.
    """
    return datasets.Dataset.from_list(data)


def read_txt_files_to_dataset(folder_path, label = 2):
    """
    Reads all .txt files in a folder (and its subfolders), extracts text, and converts to Hugging Face Dataset.
    Each .txt file contains a single text line.
    
    Args:
        folder_path (str): Path to the root folder containing .txt files.
    
    Returns:
        datasets.Dataset: Hugging Face Dataset containing text and file path information.
    """
    data = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    text = f.read().strip()
                    data.append({"text": text, "file_path": file_path, "label" : label})
    
    # Convert to Hugging Face Dataset
    #hf_dataset = datasets.Dataset.from_list(data)
    return data

## 5.2 UIT VSFC

In [63]:
test1 = pd.read_csv (UIT_VSFC_0)
test2 = pd.read_csv (UIT_VSFC_1)
test3 = pd.read_csv (UIT_VSFC_2)
uit_vsfc_pandas = pd.concat ([test1,test2,test3])
uit_vsfc_hf = datasets.Dataset.from_pandas(uit_vsfc_pandas)
test_report = evaluate_model(uit_vsfc_hf, vietnamese_model, model, english_model, text_col = "sentences", label_col = "labels", num_label = 3 )
print("Test Dataset Evaluation Report:\n", test_report)

  0%|          | 10/3166 [00:00<01:05, 48.49it/s]

100%|██████████| 3166/3166 [00:53<00:00, 58.77it/s]
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.0000    0.0000    0.0000      1409
     neutral     0.0527    1.0000    0.1002       167
    positive     0.0000    0.0000    0.0000      1590

    accuracy                         0.0527      3166
   macro avg     0.0176    0.3333    0.0334      3166
weighted avg     0.0028    0.0527    0.0053      3166



/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 5.2 VIOCD

In [66]:
from datasets import load_dataset
viocd_ds = load_dataset(VIOCD_DATA_PATH)
test_viocd = viocd_ds['test']
val_viocd = viocd_ds['validation']

In [67]:
viet_sentence_length = [len(vietnamese_tokenizer (val_viocd[i]['review'])['input_ids']) for i in range (len(val_viocd))]
pd.Series(viet_sentence_length).describe()

count    548.000000
mean      36.332117
std       29.297302
min        3.000000
25%       12.000000
50%       25.000000
75%       61.000000
max      106.000000
dtype: float64

In [68]:
# Reverse the labels for test and validation datasets
test_viocd = reverse_labels(test_viocd)
val_viocd = reverse_labels(val_viocd)

# Evaluate on test and validation datasets
test_report = evaluate_model(test_viocd, vietnamese_model, model, english_model, text_col = "review", label_col = "label", num_label = 2)
val_report = evaluate_model(val_viocd, vietnamese_model, model, english_model, text_col = "review", label_col = "label", num_label = 2)

# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)
print("Validation Dataset Evaluation Report:\n", val_report)

100%|██████████| 549/549 [00:09<00:00, 56.45it/s]
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()}

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.5082    1.0000    0.6739       279
    positive     0.0000    0.0000    0.0000       270

    accuracy                         0.5082       549
   macro avg     0.2541    0.5000    0.3370       549
weighted avg     0.2583    0.5082    0.3425       549

Validation Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.5164    1.0000    0.6811       283
    positive     0.0000    0.0000    0.0000       265

    accuracy                         0.5164       548
   macro avg     0.2582    0.5000    0.3406       548
weighted avg     0.2667    0.5164    0.3517       548




/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 5.3 VLSP

In [69]:
# Main execution
file_path = VLSP_DATA_PATH  # Update with your file path

# 1. Read and preprocess the TXT file
vlsp_data = read_txt_to_data(file_path)

# 2. Convert to Hugging Face Dataset
vlsp_dataset = create_hf_dataset(vlsp_data)

# 3. Perform Sentiment Analysis
#predictions = sentiment_analysis_on_dataset(hf_dataset, vietnamese_model, model, english_model)

val_report = evaluate_model(vlsp_dataset, vietnamese_model, model, english_model, text_col = "text", label_col = "label", num_label = 3)

# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)

  1%|          | 12/1050 [00:00<00:18, 56.52it/s]

100%|██████████| 1050/1050 [00:18<00:00, 56.67it/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.5082    1.0000    0.6739       279
    positive     0.0000    0.0000    0.0000       270

    accuracy                         0.5082       549
   macro avg     0.2541    0.5000    0.3370       549
weighted avg     0.2583    0.5082    0.3425       549




/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# 5.4 AIVIVN 2019

In [70]:
import pandas as pd
import datasets
aivivn_dataset = pd.read_csv(AIVIVN_DATA_PATH)

In [71]:
aivivn_dataset_hf = datasets.Dataset.from_pandas(aivivn_dataset)
aivivn_dataset_hf = aivivn_dataset_hf.rename_columns ({"mapped_rating" : "label"})

In [72]:
reverse_aivivn = reverse_labels(aivivn_dataset_hf)
test_report = evaluate_model(reverse_aivivn, vietnamese_model, model, english_model, text_col = "discriptions", label_col = "label", num_label = 2)
# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)

100%|██████████| 10094/10094 [02:56<00:00, 57.04it/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.4751    1.0000    0.6442      4796
    positive     0.0000    0.0000    0.0000      5298

    accuracy                         0.4751     10094
   macro avg     0.2376    0.5000    0.3221     10094
weighted avg     0.2258    0.4751    0.3061     10094




/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 5.5 ntc-scv 

In [73]:
from datasets import load_dataset
ntc_scv = load_dataset(NTC_SCV)
test_ntc_scv = ntc_scv['test']
test_ntc_scv = map_ntc_scv_label (test_ntc_scv)
# change huggingface dataset column name from sentence to text
test_ntc_scv = test_ntc_scv.rename_column("sentence", "text")
test_ntc_scv

Map: 100%|██████████| 10000/10000 [00:00<00:00, 13465.87 examples/s]


Dataset({
    features: ['text', 'label'],
    num_rows: 10000
})

In [74]:
viet_sentence_length = [len(vietnamese_tokenizer (test_ntc_scv[i]['text'])['input_ids']) for i in range (10000)]

In [75]:
pd.Series (viet_sentence_length).describe()

count    10000.00000
mean       103.78480
std         93.79315
min         10.00000
25%         44.75000
50%         78.00000
75%        129.00000
max       1228.00000
dtype: float64

In [76]:
test_report = evaluate_model(test_ntc_scv, vietnamese_model, model, english_model, text_col = "text", label_col = "label", num_label = 2)
# Print the evaluation reports
print("Test Dataset Evaluation Report:\n", test_report)

100%|██████████| 10000/10000 [03:15<00:00, 51.13it/s]

Test Dataset Evaluation Report:
               precision    recall  f1-score   support

    negative     0.5000    1.0000    0.6667      5000
    positive     0.0000    0.0000    0.0000      5000

    accuracy                         0.5000     10000
   macro avg     0.2500    0.5000    0.3333     10000
weighted avg     0.2500    0.5000    0.3333     10000




/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
